In [1]:
import logging
from concurrent import futures

from utils import log
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
from pylab import mpl

plt.style.use("seaborn-pastel")

# 字体设置
mpl.rcParams['font.family'] = ['Microsoft Yahei']  # 使用字体中的无衬线体
mpl.rcParams['font.size'] = 20  # 设置字体大小
mpl.rcParams['axes.unicode_minus'] = False  # 使坐标轴刻度标签正常显示正负号

C:\Users\lenovo\AppData\Local\Temp\ipykernel_8628\3695148398.py:3: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-pastel")


# 1 预处理

In [3]:
COLUMNS = ['name', 'id', 'level', 'num', 'all_point', 'user', 'point', 'content', 'lan', 'date', 'ip']

In [80]:
df = pd.read_excel('景点评论.xlsx', names=COLUMNS)

In [53]:
df

,name,id,level,num,all_point,user,point,content,lan,date,ip
0,西佘山园,76825,NaN,107,4.7,走走停停。。,5.0,周末带娃随便爬一下还是不错的，增进感情,zh-cn,2023-05-31,上海
1,西佘山园,76825,NaN,107,4.7,139****8030,2.0,2月初的时候带爸妈去逛了一圈，山倒不高，不过对平时缺乏运动的或者腿脚不便的上了年纪的人来说还...,zh-cn,2023-03-22,上海
2,西佘山园,76825,NaN,107,4.7,M36****7643,5.0,上海松江佘山，人少景美，空气好。,zh-cn,2023-03-22,上海
3,西佘山园,76825,NaN,107,4.7,ell****ng,5.0,在上海算是非常值得一游的地方了,zh-cn,2023-01-31,上海
4,西佘山园,76825,NaN,107,4.7,EricmengTW,4.0,Very good Very nice,zh-cn,2022-12-26,湖北
...,...,...,...,...,...,...,...,...,...,...,...
42174,海螺沟,89792,5.0,3614,4.3,ysypyq521,5.0,非常好！景区环境卫生都很好！合适家人一起游玩！,zh-cn,2019-04-06,NaN
42175,海螺沟,89792,5.0,3614,4.3,M59****86,5.0,三月份游海螺沟门票及交通车缆车全部享受半价，四号营地冰川有太阳的照下，光芒四射，有远近雪山的...,zh-cn,2019-04-06,NaN
42176,海螺沟,89792,5.0,3614,4.3,丑丑诺诺,5.0,非常值得一来看，冰川时代，雪景美,zh-cn,2019-04-06,NaN
42177,海螺沟,89792,5.0,3614,4.3,my_way66,5.0,景区管理不错，基础设施完善，沿着冰川和雪山，走在森林中,zh-cn,2019-04-06,NaN


In [54]:
df.isnull().sum()

name             0
id               0
level         9518
num              0
all_point        0
user             0
point            0
content          0
lan              0
date             0
ip           34688
dtype: int64

In [81]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

0

In [58]:
df.shape

(41190, 11)

In [82]:
df.date = pd.to_datetime(df.date)

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41190 entries, 0 to 42178
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       41190 non-null  object        
 1   id         41190 non-null  int64         
 2   level      31988 non-null  float64       
 3   num        41190 non-null  int64         
 4   all_point  41190 non-null  float64       
 5   user       41190 non-null  object        
 6   point      41190 non-null  float64       
 7   content    41190 non-null  object        
 8   lan        41190 non-null  object        
 9   date       41190 non-null  datetime64[ns]
 10  ip         7129 non-null   object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 3.8+ MB


In [61]:
df_save = df.copy()

# 2 分析
## 2.1 清洗 content
### 2.1.1 去掉提取错误的

In [83]:
drop_index = []
for i, j in tqdm(enumerate(df['content'])):
    if '{' in j:
        drop_index.append(i)

41051it [00:00, 1965619.13it/s]


In [84]:
df.drop(drop_index, inplace=True)

In [87]:
df.index = np.arange(df.shape[0])

In [96]:
df['key'] = ''

In [97]:
df

,name,id,level,num,all_point,user,point,content,lan,date,ip,key
0,西佘山园,76825,NaN,107,4.7,走走停停。。,5.0,周末带娃随便爬一下还是不错的，增进感情,zh-cn,2023-05-31,上海,
1,西佘山园,76825,NaN,107,4.7,139****8030,2.0,2月初的时候带爸妈去逛了一圈，山倒不高，不过对平时缺乏运动的或者腿脚不便的上了年纪的人来说还...,zh-cn,2023-03-22,上海,
2,西佘山园,76825,NaN,107,4.7,M36****7643,5.0,上海松江佘山，人少景美，空气好。,zh-cn,2023-03-22,上海,
3,西佘山园,76825,NaN,107,4.7,ell****ng,5.0,在上海算是非常值得一游的地方了,zh-cn,2023-01-31,上海,
4,西佘山园,76825,NaN,107,4.7,EricmengTW,4.0,Very good Very nice,zh-cn,2022-12-26,湖北,
...,...,...,...,...,...,...,...,...,...,...,...,...
40964,海螺沟,89792,5.0,3614,4.3,ysypyq521,5.0,非常好！景区环境卫生都很好！合适家人一起游玩！,zh-cn,2019-04-06,NaN,
40965,海螺沟,89792,5.0,3614,4.3,M59****86,5.0,三月份游海螺沟门票及交通车缆车全部享受半价，四号营地冰川有太阳的照下，光芒四射，有远近雪山的...,zh-cn,2019-04-06,NaN,
40966,海螺沟,89792,5.0,3614,4.3,丑丑诺诺,5.0,非常值得一来看，冰川时代，雪景美,zh-cn,2019-04-06,NaN,
40967,海螺沟,89792,5.0,3614,4.3,my_way66,5.0,景区管理不错，基础设施完善，沿着冰川和雪山，走在森林中,zh-cn,2019-04-06,NaN,


## 2.2 关键词提取

In [2]:
import time
import urllib.request
import urllib.parse
import json
import hashlib
import base64
#接口地址
url ="http://ltpapi.xfyun.cn/v1/ke"
#开放平台应用ID
x_appid = "e933ef2e"
#开放平台应用接口秘钥
api_key = "f03cc6348cc50aa841ddeae253700f7f"
#语言文本
# TEXT="汉皇重色思倾国，御宇多年求不得。杨家有女初长成，养在深闺人未识。天生丽质难自弃，一朝选在君王侧。"


def ltp(TEXT):
    body = urllib.parse.urlencode({'text': TEXT}).encode('utf-8')
    param = {"type": "dependent"}
    x_param = base64.b64encode(json.dumps(param).replace(' ', '').encode('utf-8'))
    x_time = str(int(time.time()))
    x_checksum = hashlib.md5(api_key.encode('utf-8') + str(x_time).encode('utf-8') + x_param).hexdigest()
    x_header = {'X-Appid': x_appid,
                'X-CurTime': x_time,
                'X-Param': x_param,
                'X-CheckSum': x_checksum}
    req = urllib.request.Request(url, body, x_header)
    result = urllib.request.urlopen(req)
    result = result.read()
    result = result.decode('utf-8')
    result = json.loads(result)

    if result['desc'] == 'success':
        return result['data']['ke']
#
#
# if __name__ == '__main__':
#     xx = ltp()

In [3]:
ltp('公园绿化很好的，空气清新。适合散心')

[{'score': '0.692', 'word': '散心'},
 {'score': '0.674', 'word': '公园'},
 {'score': '0.669', 'word': '绿化'},
 {'score': '0.635', 'word': '空气'},
 {'score': '0.607', 'word': '清新'},
 {'score': '0.599', 'word': '适合'}]

In [100]:
log.config_logging('key_log.log')

In [116]:
def fafa_ltp(row_num):
    if not df.loc[row_num, 'key'] or "'count':" in df.loc[row_num, 'key']:
        res = str(ltp(df.loc[row_num, 'content']))
        logging.debug(f'row={row_num} ltp={res}')
        df.loc[row_num, 'key'] = res

In [ ]:
tasks = []
with futures.ThreadPoolExecutor(15) as t:

    for row in tqdm(df.index):
        tasks.append(t.submit(fafa_ltp, row))

    for task in tqdm(futures.as_completed(tasks), total=len(tasks)):
        task.result()


 31%|███       | 12521/40969 [01:42<03:54, 121.57it/s]


In [114]:
df.loc[:, ['content', 'key']]

,content,key
0,周末带娃随便爬一下还是不错的，增进感情,"[{'count': 1, 'eq': '0.953', 'final': '0.671',..."
1,2月初的时候带爸妈去逛了一圈，山倒不高，不过对平时缺乏运动的或者腿脚不便的上了年纪的人来说还...,"[{'score': '0.575', 'word': '钢钉'}, {'score': '..."
2,上海松江佘山，人少景美，空气好。,"[{'score': '0.686', 'word': '佘山'}, {'score': '..."
3,在上海算是非常值得一游的地方了,"[{'score': '0.624', 'word': '地方'}, {'score': '..."
4,Very good Very nice,None
...,...,...
40964,非常好！景区环境卫生都很好！合适家人一起游玩！,"[{'score': '0.652', 'word': '景区'}, {'score': '..."
40965,三月份游海螺沟门票及交通车缆车全部享受半价，四号营地冰川有太阳的照下，光芒四射，有远近雪山的...,"[{'score': '0.641', 'word': '螺沟'}, {'score': '..."
40966,非常值得一来看，冰川时代，雪景美,"[{'score': '0.749', 'word': '雪景'}, {'score': '..."
40967,景区管理不错，基础设施完善，沿着冰川和雪山，走在森林中,"[{'score': '0.663', 'word': '雪山'}, {'score': '..."


In [115]:
df.to_excel('备份.xlsx')